In [30]:
import numpy as np
import random
import scipy.integrate as scp
import numpy.random as rnd
import time
import matplotlib.pyplot as plt
import numpy.random as rnd
import copy
from matplotlib.pyplot import figure
from pyomo.environ import *
from pyomo.dae import *
import pandas as pd
import pickle
import scipy.stats
from math import comb
from numpy.polynomial import polynomial as P
from scipy.integrate import odeint
from pyomo.environ import ConstraintList

eps  = np.finfo(float).eps

In [31]:

''' Data treatment'''
def save_pkl(item, fname):
    sn = 'tmp7/' + fname
    with open(sn, 'wb') as handle:
        pickle.dump(item, handle) #, protocol=pickle.HIGHEST_PROTOCOL)
    print(f'File saved at: {sn}')
    return None
# return None

def load_pkl(fname):
    with open(fname, 'rb') as handle:
        ans = pickle.load(handle)
    print(f'Loaded from: {fname}')
    return ans

def to_dict(x,dt):
    obs = list(x)
    # tp = list(time)
    dicx = {}
    for i in range(len(x)):
        dicx[dt*i] = obs[i]            # change thye value of 38.4
    return dicx

def get_grad(x, t):
    dxdt = [[],[],[],[]]
    for n in range(x.shape[0]):
        for i in range(len(x[0]) - 1):
            dxdt[n].append((x[n][i + 1] - x[n][i])/(t[i + 1] - t[i]))
        dxdt[n].append(dxdt[n][-1])
    return dxdt


In [32]:
xobs1 = load_pkl('data25/xobs1.pkl')
xobs2 = load_pkl('data25/xobs2.pkl')
xobs3 = load_pkl('data25/xobs3.pkl')
xobs4 = load_pkl('data25/xobs4.pkl')
mu_con   = load_pkl('data25/u_con.pkl')
tf_N  = load_pkl('data25/tf_N.pkl')
N_in  = load_pkl('data25/N_in.pkl')
operation_con = load_pkl('data25/operation_con.pkl')


tt1 = load_pkl('data25/tt1.pkl')
std_value1 = load_pkl('data25/std_value1.pkl')
std_value2 = load_pkl('data25/std_value2.pkl')
std_value3 = load_pkl('data25/std_value3.pkl')
std_value4 = load_pkl('data25/std_value4.pkl')
data_init = load_pkl('data25/data_init.pkl')

u_dic1    = load_pkl('data25/u_dic1.pkl')
u_dic2    = load_pkl('data25/u_dic2.pkl')
u_dic3    = load_pkl('data25/u_dic3.pkl')
u_dic4    = load_pkl('data25/u_dic4.pkl')

operation_con1 = operation_con[0]
operation_con2 = operation_con[1]
operation_con3 = operation_con[2]
operation_con4 = operation_con[3]

mu1 = mu_con[0] #u contains FNin and I0
mu2 = mu_con[1] 
mu3 = mu_con[2] 
mu4 = mu_con[3] 

tf    = 240
# tf    = 16*24
steps_= 24
dt    = tf/steps_ #dt is fixed





Loaded from: data25/xobs1.pkl
Loaded from: data25/xobs2.pkl
Loaded from: data25/xobs3.pkl
Loaded from: data25/xobs4.pkl
Loaded from: data25/u_con.pkl
Loaded from: data25/tf_N.pkl
Loaded from: data25/N_in.pkl
Loaded from: data25/operation_con.pkl
Loaded from: data25/tt1.pkl
Loaded from: data25/std_value1.pkl
Loaded from: data25/std_value2.pkl
Loaded from: data25/std_value3.pkl
Loaded from: data25/std_value4.pkl
Loaded from: data25/data_init.pkl
Loaded from: data25/u_dic1.pkl
Loaded from: data25/u_dic2.pkl
Loaded from: data25/u_dic3.pkl
Loaded from: data25/u_dic4.pkl


In [33]:
xobs1.shape

(4, 25)

In [34]:
operation_con

array([[15.  ],
       [37.5 ],
       [48.75],
       [26.25]])

In [35]:
u_dic1

{0.0: 50.0,
 10.0: 50.0,
 20.0: 50.0,
 30.0: 50.0,
 40.0: 50.0,
 50.0: 50.0,
 60.0: 50.0,
 70.0: 125.0,
 80.0: 125.0,
 90.0: 125.0,
 100.0: 125.0,
 110.0: 125.0,
 120.0: 125.0,
 130.0: 162.5,
 140.0: 162.5,
 150.0: 162.5,
 160.0: 162.5,
 170.0: 162.5,
 180.0: 162.5,
 190.0: 87.5,
 200.0: 87.5,
 210.0: 87.5,
 220.0: 87.5,
 230.0: 87.5,
 240.0: 87.5}

In [36]:
x_max = np.max(np.array([xobs1[0],xobs2[0],xobs3[0],xobs4[0]]))
q_max = np.max(np.array([xobs1[2],xobs2[2],xobs3[2],xobs4[2]]))
I_max = np.max(np.concatenate(mu_con))

In [37]:
I_max 

190.625

In [38]:

num_N = 4
tf_N  = tf/(num_N)
dstep_N = int(tf_N/dt)

def get_grad(x, t):
    dxdt = [[],[],[],[]]
    for n in range(x.shape[0]):
        for i in range(len(x[0]) - 1):
            dxdt[n].append((x[n][i + 1] - x[n][i])/(t[i + 1] - t[i]))
        dxdt[n].append(dxdt[n][-1])
    return dxdt

Xt1   = [to_dict(xobs1[0],dt), to_dict(xobs1[1],dt), to_dict(xobs1[2],dt), to_dict(xobs1[3],dt)] 
dXdt1 = [to_dict(get_grad(xobs1, tt1)[0],dt),to_dict(get_grad(xobs1, tt1)[1],dt),to_dict(get_grad(xobs1, tt1)[2],dt),to_dict(get_grad(xobs1, tt1)[3],dt)]
Xt2   = [to_dict(xobs2[0],dt), to_dict(xobs2[1],dt), to_dict(xobs2[2],dt), to_dict(xobs2[3],dt)] 
dXdt2 = [to_dict(get_grad(xobs2, tt1)[0],dt),to_dict(get_grad(xobs2, tt1)[1],dt),to_dict(get_grad(xobs2, tt1)[2],dt),to_dict(get_grad(xobs2, tt1)[3],dt)]
Xt3   = [to_dict(xobs3[0],dt), to_dict(xobs3[1],dt), to_dict(xobs3[2],dt), to_dict(xobs3[3],dt)] 
dXdt3 = [to_dict(get_grad(xobs3, tt1)[0],dt),to_dict(get_grad(xobs3, tt1)[1],dt),to_dict(get_grad(xobs3, tt1)[2],dt),to_dict(get_grad(xobs3, tt1)[3],dt)]
Xt4   = [to_dict(xobs4[0],dt), to_dict(xobs4[1],dt), to_dict(xobs4[2],dt), to_dict(xobs4[3],dt)] 
dXdt4 = [to_dict(get_grad(xobs4, tt1)[0],dt),to_dict(get_grad(xobs4, tt1)[1],dt),to_dict(get_grad(xobs4, tt1)[2],dt),to_dict(get_grad(xobs4, tt1)[3],dt)]
# Xt5   = [to_dict(xobs5[0],dt), to_dict(xobs5[1],dt), to_dict(xobs5[2],dt), to_dict(xobs5[3],dt)] 
# dXdt5 = [to_dict(get_grad(xobs5, tt1)[0],dt),to_dict(get_grad(xobs5, tt1)[1],dt),to_dict(get_grad(xobs5, tt1)[2],dt),to_dict(get_grad(xobs5, tt1)[3],dt)]



In [39]:
Xt1[0]

{0.0: 0.18,
 10.0: 0.22282356533121636,
 20.0: 0.3884461274741864,
 30.0: 0.4731542776451567,
 40.0: 0.6082363901277796,
 50.0: 0.6905292550920789,
 60.0: 0.7986625995832054,
 70.0: 0.9156086295508562,
 80.0: 1.0005432081109003,
 90.0: 1.0432792294587738,
 100.0: 1.1609616589706404,
 110.0: 1.3296742728610913,
 120.0: 1.258933184531873,
 130.0: 1.334642899263699,
 140.0: 1.3191513184701842,
 150.0: 1.5040757024460516,
 160.0: 1.4035924807576456,
 170.0: 1.4055978557504674,
 180.0: 1.411345072196402,
 190.0: 1.497992527428832,
 200.0: 1.3854147381264657,
 210.0: 1.4512474037079868,
 220.0: 1.494099905690478,
 230.0: 1.4543616599667977,
 240.0: 1.5057060910756397}

In [40]:
N1 = operation_con[0][0]
N2 = operation_con[1][0]
N3 = operation_con[2][0]
N4 = operation_con[3][0]

In [41]:
mu1[0][0]

50.0

In [42]:
mu1.T[0]

array([ 50. , 125. , 162.5,  87.5])

In [43]:
# Fcn1 = mu1.T[0]

I1 = mu1.T[0]

In [44]:
I1[0]

50.0

In [45]:
# eff_neuron = 7# actually applied number of neurons
no_euron = 7 # total number of neurons

In [46]:
u_dic1

{0.0: 50.0,
 10.0: 50.0,
 20.0: 50.0,
 30.0: 50.0,
 40.0: 50.0,
 50.0: 50.0,
 60.0: 50.0,
 70.0: 125.0,
 80.0: 125.0,
 90.0: 125.0,
 100.0: 125.0,
 110.0: 125.0,
 120.0: 125.0,
 130.0: 162.5,
 140.0: 162.5,
 150.0: 162.5,
 160.0: 162.5,
 170.0: 162.5,
 180.0: 162.5,
 190.0: 87.5,
 200.0: 87.5,
 210.0: 87.5,
 220.0: 87.5,
 230.0: 87.5,
 240.0: 87.5}

In [48]:
model         = AbstractModel()
model.t       = ContinuousSet(bounds=[0, tt1[-1]])
model.tm      = Set(within=model.t)
model.x1_noise = Param(model.tm)
model.n1_noise = Param(model.tm)
model.q1_noise = Param(model.tm)
model.f1_noise = Param(model.tm)

model.x1 = Var(model.t, within=PositiveReals,initialize=Xt1[0]) 
model.n1 = Var(model.t, within=PositiveReals,initialize=Xt1[1])
model.q1 = Var(model.t, within=PositiveReals,initialize=Xt1[2])
model.f1 = Var(model.t, within=PositiveReals,initialize=Xt1[3])



def _I1_init(model, t):
    if t <= tf_N*1:
        return I1[0]
    elif tf_N*1 < t <= tf_N*2:
        return I1[1]
    elif tf_N*2 < t <= tf_N*3:
        return I1[2]
    else:
        return I1[3]
    

model.I1 = Param(model.t,initialize=u_dic1,default=_I1_init)
    
model.I = RangeSet(no_euron) # define number of neurons
model.J = RangeSet(3) # number of inputs
model.k = RangeSet(1)
init_dict_w1 = {(1,1):np.random.normal(0, 1)}
init_dict_b1 = {(1,1):np.random.normal(0, 1)}
init_dict_w2 = {(1,1):np.random.normal(0, 1)}


for i in range(1,no_euron+1): # range of the number of neuron
    for j in range(1,4):# range of the number of inputs
        # print (i,j)
        init_dict_ij = {(i,j):np.random.normal(0, 0.1)}
        init_dict_w1.update(init_dict_ij)
model.w1 = Var(model.I,model.J, initialize=init_dict_w1,bounds = (-1,1))


for i in range(1,2): # for 1 layer this is fixed
    for j in range(1,no_euron+1): # range of the number of neuron
        # print (i,j)
        init_dict_ij = {(i,j):np.random.normal(0, 0.1)}
        init_dict_b1.update(init_dict_ij)
model.b1 = Var(model.k,model.I,initialize=init_dict_b1,bounds = (-1,1))

for i in range(1,2):# for 1 layer this is fixed
    for j in range(1,no_euron+1):# range of the number of neuron
        # print (i,j)
        init_dict_ij = {(i,j):np.random.normal(0, 0.1)}
        init_dict_w2.update(init_dict_ij)
model.w2           = Var(model.k,model.I,initialize=init_dict_w2,bounds = (-1,1))
model.b2           = Var(domain = Reals,initialize=0,bounds = (-1,1))

model.node1 = Var(model.k,model.I,model.t,initialize=0.1,bounds = (-1,1))


model.ud               = Var(domain = Reals, bounds=(0,0.1),  initialize=0)
model.un               = Var(domain = Reals, bounds=(0,5),  initialize=2.8)
model.kn               = Var(domain = Reals, bounds=(0,2),  initialize=1.4)
model.theta            = Var(domain = Reals, bounds=(5,10),  initialize=7.5)
model.gamma            = Var(domain = Reals, bounds=(5,10),  initialize=8.4)
model.epsilon          = Var(domain = Reals, bounds=(0,1),  initialize=0.1)

model.u1     = Var(model.t,domain = NonNegativeReals, bounds=(0,0.1),initialize=0.05)

model.x1dot = DerivativeVar(model.x1, wrt=model.t,domain = Reals,initialize = dXdt1[0])
model.n1dot = DerivativeVar(model.n1, wrt=model.t,domain = Reals,initialize = dXdt1[1])
model.q1dot = DerivativeVar(model.q1, wrt=model.t,domain = Reals,initialize = dXdt1[2])
model.f1dot = DerivativeVar(model.f1, wrt=model.t,domain = Reals,initialize = dXdt1[3])


# EXP1 ------------------------------------
def NN_node11_exp1(model,t):

    return model.node1[1,1,t] == tanh(model.x1[t]/x_max*model.w1[1,1]+model.q1[t]/q_max*model.w1[1,2]+ model.I1[t]/I_max *model.w1[1,3] +model.b1[1,1])
model.NN_node11_exp1_cons = Constraint(model.t, rule = NN_node11_exp1)

def NN_node12_exp1(model,t):

    return model.node1[1,2,t] == tanh(model.x1[t]/x_max*model.w1[2,1]+model.q1[t]/q_max*model.w1[2,2]+ model.I1[t]/I_max *model.w1[2,3] +model.b1[1,2])
model.NN_node12_exp1_cons = Constraint(model.t, rule = NN_node12_exp1)

def NN_node13_exp1(model,t):
    return model.node1[1,3,t] == tanh(model.x1[t]/x_max*model.w1[3,1]+model.q1[t]/q_max*model.w1[3,2]+ model.I1[t]/I_max *model.w1[3,3] + model.b1[1,3])

model.NN_node13_exp1_cons = Constraint(model.t, rule = NN_node13_exp1)

def NN_node14_exp1(model,t):
    return model.node1[1,4,t] == tanh(model.x1[t]/x_max*model.w1[4,1]+model.q1[t]/q_max*model.w1[4,2]+ model.I1[t]/I_max *model.w1[4,3] + model.b1[1,4])

model.NN_node14_exp1_cons = Constraint(model.t, rule = NN_node14_exp1)

def NN_node15_exp1(model,t):
    return model.node1[1,5,t] == tanh(model.x1[t]/x_max*model.w1[5,1]+model.q1[t]/q_max*model.w1[5,2]+ model.I1[t]/I_max *model.w1[5,3] + model.b1[1,5])

model.NN_node15_exp1_cons = Constraint(model.t, rule = NN_node15_exp1)

def NN_node16_exp1(model,t):
    return model.node1[1,6,t] == tanh(model.x1[t]/x_max*model.w1[6,1]+model.q1[t]/q_max*model.w1[6,2]+ model.I1[t]/I_max *model.w1[6,3] + model.b1[1,6])

model.NN_node16_exp1_cons = Constraint(model.t, rule = NN_node16_exp1)

def NN_node17_exp1(model,t):
    return model.node1[1,7,t] == tanh(model.x1[t]/x_max*model.w1[7,1]+model.q1[t]/q_max*model.w1[7,2]+ model.I1[t]/I_max *model.w1[7,3] + model.b1[1,7])

model.NN_node17_exp1_cons = Constraint(model.t, rule = NN_node17_exp1)


def u1_con_exp1(model,t):
    return model.u1[t] == model.node1[1,1,t]*model.w2[1,1] + model.node1[1,2,t]*model.w2[1,2] + model.node1[1,3,t]*model.w2[1,3] + model.node1[1,4,t]*model.w2[1,4] + model.node1[1,5,t]*model.w2[1,5] + model.node1[1,6,t]*model.w2[1,6] + model.node1[1,7,t]*model.w2[1,7] + model.b2
model.u1_con_exp1  =Constraint(model.t,rule = u1_con_exp1)

# def u1_con_exp1(model,t):
#     return model.u1[t] == model.node1[1,1,t]*model.w2[1,1]+ model.node1[1,2,t]*model.w2[1,2] + model.b2
# model.u1_con_exp1  =Constraint(model.t,rule = u1_con_exp1)


def dxdt1(model,t):
    if t == 0:
        return Constraint.Skip
    return model.x1dot[t] == model.u1[t] * model.x1[t] - model.ud * model.x1[t]
model.dxdtcon1 = Constraint(model.t, rule = dxdt1)


def dndt1(model,t):
    if t == 0:
        return Constraint.Skip
    return model.n1dot[t] == -model.un*(model.n1[t]/(model.n1[t]+model.kn))*model.x1[t]
model.dndtcon1 = Constraint(model.t, rule = dndt1)

def dqdt1(model,t):
    if t == 0:
        return Constraint.Skip
    return model.q1dot[t] == model.un*(model.n1[t]/(model.n1[t]+model.kn)) - (model.u1[t]-model.ud)*model.q1[t]
model.dqdtcon1 = Constraint(model.t, rule = dqdt1)


def dfdt1(model,t):
    if t == 0:
        return Constraint.Skip
    return model.f1dot[t] == model.u1[t]*(model.theta*model.q1[t]-model.epsilon*model.f1[t]) - model.gamma*model.un*(model.n1[t]/(model.n1[t]+model.kn)) + model.ud*model.epsilon*model.f1[t]
model.dfdtcon1 = Constraint(model.t, rule = dfdt1)


number_datapoints1 = xobs1.shape[1]
number_datapoints2 = xobs2.shape[1]
number_datapoints3 = xobs3.shape[1]
number_datapoints4 = xobs4.shape[1]


number_spc1 = xobs1.shape[0]
number_spc2 = xobs2.shape[0]
number_spc3 = xobs3.shape[0]
number_spc4 = xobs4.shape[0]

def obj(model):

 

    variance1    = (sum((model.x1[t]-model.x1_noise[t])**2 for t in model.tm)+sum((model.n1[t]-model.n1_noise[t])**2 for t in model.tm)+sum((model.q1[t]-model.q1_noise[t])**2 for t in model.tm)+sum((model.f1[t]-model.f1_noise[t])**2 for t in model.tm))/(number_datapoints1 * number_spc1)


    obj1 = (sum((model.x1[t]-model.x1_noise[t])**2 for t in model.tm)+sum((model.n1[t]-model.n1_noise[t])**2 for t in model.tm)+sum((model.q1[t]-model.q1_noise[t])**2 for t in model.tm)+sum((model.f1[t]-model.f1_noise[t])**2 for t in model.tm))/2/(variance1+1e-12) - (number_datapoints1 * number_spc1)*log(1/(sqrt(2*3.14159*(variance1+1e-12))))

    # # obj2 = (sum((model.x2[t]-model.x2_noise[t])**2 for t in model.tm)+sum((model.n2[t]-model.n2_noise[t])**2 for t in model.tm)+sum((model.q2[t]-model.q2_noise[t])**2 for t in model.tm)+sum((model.f2[t]-model.f2_noise[t])**2 for t in model.tm))/2/(variance2+1e-12) - (number_datapoints2 * number_spc2)*log(1/(sqrt(2*3.14159*(variance2+1e-12))))

    # # obj3 = (sum((model.x3[t]-model.x3_noise[t])**2 for t in model.tm)+sum((model.n3[t]-model.n3_noise[t])**2 for t in model.tm)+sum((model.q3[t]-model.q3_noise[t])**2 for t in model.tm)+sum((model.f3[t]-model.f3_noise[t])**2 for t in model.tm))/2/(variance3+1e-12) - (number_datapoints3 * number_spc3)*log(1/(sqrt(2*3.14159*(variance3+1e-12))))

    # # obj4 = (sum((model.x4[t]-model.x4_noise[t])**2 for t in model.tm)+sum((model.n4[t]-model.n4_noise[t])**2 for t in model.tm)+sum((model.q4[t]-model.q4_noise[t])**2 for t in model.tm)+sum((model.f4[t]-model.f4_noise[t])**2 for t in model.tm))/2/(variance4+1e-12) - (number_datapoints4 * number_spc4)*log(1/(sqrt(2*3.14159*(variance4+1e-12))))


    # return sum((model.x1[t]-model.x1_noise[t])**2 for t in model.tm)+sum((model.n1[t]-model.n1_noise[t])**2 for t in model.tm)+sum((model.q1[t]-model.q1_noise[t])**2 for t in model.tm)+sum((model.f1[t]-model.f1_noise[t])**2 for t in model.tm)

    return obj1

model.obj = Objective(rule=obj)


instance = model.create_instance(data_init)
instance.x1[0].fix(0.18)
instance.n1[0].fix(N1)
instance.q1[0].fix(80)
instance.f1[0].fix(120)


discretizer = TransformationFactory('dae.collocation')
discretizer.apply_to(instance,nfe=30,ncp=3,wrt=instance.t,scheme='LAGRANGE-RADAU')


    # fix initial value

solver=SolverFactory('ipopt')
solver.options['max_iter'] = 100000
solver.options['tol'] = 1e-9
# solver.options['print_level'] = 5  # Adjust based on your needs

# # solver.options['max_cpu_time'] = 600  # Limit solver CPU time to 10 minutes
# solver.options['print_level'] = 10    # Increase verbosity for more detailed log output
# solver.options['hessian_approximation'] = 'limited-memory'  # Useful for large-scale problems

# results = solver.solve(instance, tee=True, logfile='solver_log.txt')
results = solver.solve(instance, tee=True)


Ipopt 3.12.13: max_iter=100000
tol=1e-09


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    10119
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:    12088

Total number of variables............................:     1490
                     variables with only lower bounds:      360
                variables with lower and upper bounds:      770
                     variables with only 

In [50]:
instance.I1.display()

I1 : Size=91, Index=t, Domain=Any, Default=(function), Mutable=False
    Key   : Value
      0.0 :  50.0
     10.0 :  50.0
     20.0 :  50.0
     30.0 :  50.0
     40.0 :  50.0
     50.0 :  50.0
     60.0 :  50.0
     70.0 : 125.0
     80.0 : 125.0
     90.0 : 125.0
    100.0 : 125.0
    110.0 : 125.0
    120.0 : 125.0
    130.0 : 162.5
    140.0 : 162.5
    150.0 : 162.5
    160.0 : 162.5
    170.0 : 162.5
    180.0 : 162.5
    190.0 :  87.5
    200.0 :  87.5
    210.0 :  87.5
    220.0 :  87.5
    230.0 :  87.5
    240.0 :  87.5
